In [24]:
!python database_helpers.py

d:\Zeug\Unizeug\Master_DataScience\3.Semester\Urban Technologies\Project\database_helpers.py:11: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [56]:
import pandas as pd
import numpy as np
from database_helpers import ForeclosureObjectSchema, ForeclosureCaseSchema, engine, session

In [26]:
cases_df = pd.read_sql(session.query(ForeclosureCaseSchema).statement, engine)
cases_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4637 entries, 0 to 4636
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   link                     4637 non-null   object
 1   bundesland_code          4637 non-null   object
 2   aktenzeichen             4637 non-null   object
 3   letzte_aktualisierung    4637 non-null   object
 4   art_der_versteigerung    4637 non-null   object
 5   grundbuch                4273 non-null   object
 6   objekt_lage              4637 non-null   object
 7   beschreibung             4637 non-null   object
 8   verkehrswert             4637 non-null   object
 9   termin                   4637 non-null   int64 
 10  ort_der_versteigerung    4637 non-null   object
 11  amtliche_bekanntmachung  4519 non-null   object
 12  exposee                  2111 non-null   object
 13  gutachten                2353 non-null   object
 14  foto                     1025 non-null  

In [27]:
objects_df = pd.read_sql(session.query(ForeclosureObjectSchema).statement, engine)
objects_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1708 entries, 0 to 1707
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    1708 non-null   int64  
 1   foreclosurecase_link  1708 non-null   object 
 2   flaeche               1708 non-null   int64  
 3   verkehrswert          1708 non-null   int64  
 4   typ                   1708 non-null   object 
 5   baujahr               731 non-null    float64
 6   raeume                281 non-null    float64
 7   raum_typen            1708 non-null   object 
 8   balkon                1708 non-null   bool   
 9   garten                1708 non-null   bool   
dtypes: bool(2), float64(2), int64(3), object(3)
memory usage: 110.2+ KB


In [28]:
objects_df.head()

,id,foreclosurecase_link,flaeche,verkehrswert,typ,baujahr,raeume,raum_typen,balkon,garten
0,1,https://www.zvg-portal.de/index.php?button=sho...,64,283000,Anderes,1959.0,3.0,"Wohnzimmer,Küche,Badezimmer",False,True
1,2,https://www.zvg-portal.de/index.php?button=sho...,0,0,Anderes,NaN,NaN,,False,False
2,3,https://www.zvg-portal.de/index.php?button=sho...,0,0,Anderes,NaN,NaN,,False,False
3,4,https://www.zvg-portal.de/index.php?button=sho...,0,0,Haus,1890.0,NaN,,False,False
4,5,https://www.zvg-portal.de/index.php?button=sho...,0,0,Haus,1890.0,NaN,,False,False


In [29]:
cases_df.head()

,link,bundesland_code,aktenzeichen,letzte_aktualisierung,art_der_versteigerung,grundbuch,objekt_lage,beschreibung,verkehrswert,termin,ort_der_versteigerung,amtliche_bekanntmachung,exposee,gutachten,foto
0,https://www.zvg-portal.de/index.php?button=sho...,bw,K 0014/ 2022,1724155560,Zwangsversteigerung zum Zwecke der Aufhebung d...,"Grundbuch von Fichtenau-Wildenstein, je in Erb...","Einfamilienhaus, unbebautes Grundstück: Haupts...","lfd. Nr. 1: unbebautes, nicht erschlossenes Gr...","lfd. Nr. 1: 21.000,00 €lfd. Nr. 2: 164.000,00 €",1729501200,"Amtsgericht Crailsheim, Schlossplatz 1, 74564 ...",None,None,None,None
1,https://www.zvg-portal.de/index.php?button=sho...,bw,K 0003/ 2023,1724664240,Zwangsversteigerung zum Zwecke der Aufhebung d...,"Mergentheim, Gemarkung Mergentheim, Flurstück ...","Sonstiges: Neunkircher Straße 89, 97980 Bad M...",Grundstück bebaut mit zweigeschossigem Reihenm...,"155.000,00 €",1729512000,"Amtsgericht Crailsheim, Schlossplatz 1, 74564 ...",None,None,None,None
2,https://www.zvg-portal.de/index.php?button=sho...,bw,0002 K 0032/ 2023,1729148100,Versteigerung im Wege der Zwangsvollstreckung,"Bad Schussenried, Blatt 5090 BV 1","Gewerbehalle: Friedrich-List-Straße 4, 88427 B...",Objektbeschreibung/Lage (lt Angabe d. Sachvers...,"334.000,00",1729580400,"Sitzungssaal des Rathauses, Wilhelm-Schussen-S...",None,None,None,None
3,https://www.zvg-portal.de/index.php?button=sho...,bw,0002 K 0048/ 2021,1725951000,Zwangsversteigerung zum Zwecke der Aufhebung d...,"Waldsee, Blatt 4249","Einfamilienhaus: Anton-Bruckner-Straße 18, 088...",lfd.Nr.\n \n \nGemarkung\n \n \nFlurstück\n \n...,"Lfd. Nr. 1: 495.000,00 € \nLfd. Nr. 2: 8.600,0...",1729666800,"Amtsgericht Ravensburg, Außenstelle Kirchstr. ...",None,None,None,None
4,https://www.zvg-portal.de/index.php?button=sho...,bw,0003 K 0130/ 2023,1727164560,Zwangsversteigerung zum Zwecke der Aufhebung d...,"Gemarkung Ketsch, Blatt 1.101","Einfamilienhaus: Hockenheimer Straße 63, 00000...","Grundstück mit 513 qm in Ketsch, Hockenheimer ...","650.000,00",1729753200,"Amtsgericht Mannheim, Zweigstelle A 2,1 - Saal...",?button=showAnhang&land_abk=bw&file_id=5543&zv...,None,None,None


In [57]:
bundesland_mapping = {
    'bw': 'Baden-Württemberg',
    'by': 'Bayern',
    'be': 'Berlin',
    'br': 'Brandenburg',
    'hb': 'Bremen',
    'hh': 'Hamburg',
    'he': 'Hessen',
    'mv': 'Mecklenburg-Vorpommern',
    'ni': 'Niedersachsen',
    'nw': 'Nordrhein-Westfalen',
    'rp': 'Rheinland-Pfalz',
    'sl': 'Saarland',
    'sn': 'Sachsen',
    'st': 'Sachsen-Anhalt',
    'sh': 'Schleswig-Holstein',
    'th': 'Thüringen',
}

cases_df = pd.read_sql(session.query(ForeclosureCaseSchema).statement, engine)
objects_df = pd.read_sql(session.query(ForeclosureObjectSchema).statement, engine)
merged_df = objects_df.merge(cases_df.drop("verkehrswert", axis= 1), how='left', left_on='foreclosurecase_link', right_on='link')
merged_df['bundesland_name'] = merged_df['bundesland_code'].map(bundesland_mapping)
filtered_df = merged_df[[
    'foreclosurecase_link', 'flaeche', 'verkehrswert', 'typ', 'baujahr',
    'raeume', 'raum_typen', 'balkon', 'garten', 'objekt_lage', 'aktenzeichen',
    'termin', 'art_der_versteigerung', 'beschreibung', 'bundesland_name'
]]
filtered_df.loc[filtered_df['raum_typen'].str.strip() == '', 'raum_typen'] = np.nan

In [58]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1708 entries, 0 to 1707
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   foreclosurecase_link   1708 non-null   object 
 1   flaeche                1708 non-null   int64  
 2   verkehrswert           1708 non-null   int64  
 3   typ                    1708 non-null   object 
 4   baujahr                731 non-null    float64
 5   raeume                 281 non-null    float64
 6   raum_typen             254 non-null    object 
 7   balkon                 1708 non-null   bool   
 8   garten                 1708 non-null   bool   
 9   objekt_lage            1708 non-null   object 
 10  aktenzeichen           1708 non-null   object 
 11  termin                 1708 non-null   int64  
 12  art_der_versteigerung  1708 non-null   object 
 13  beschreibung           1708 non-null   object 
 14  bundesland_name        1708 non-null   object 
dtypes: b

In [60]:
filtered_df.to_csv("data/zwangsversteigerungen.csv", index=False)

In [61]:
df = pd.read_csv("data/zwangsversteigerungen.csv")

In [63]:
assert filtered_df.equals(df)